In [1]:
import sys, os, json, pickle, gzip, copy, torch, numbers, tqdm
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from src.utils import ActiveElements, MetalElements, Element, AllElements
from src.utils import composit_parser
from src.feature import composition_to_feature

from pymatgen.core import Composition, Element

from scipy.stats import gaussian_kde
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR

In [2]:
from src.data import ReactionGraphDataset, ReactionDataset
from src.networks import GraphCVAE, CVAE, GraphConvolutionBlock, GraphAttentionBlock
from src.trainer import VAETrainer, BaseTrainer

GDSM = ReactionGraphDataset(feat_type='cgcnn')
GDSM.from_file('../data/screened_conditional_reaction.pkl.gz', 
               heat_temp_key=('heat_temp','median'))


In [6]:
rxn_id = []
temp_1 = []
temp_2 = []
for i, data in enumerate(GDSM):
    rxn_id.append([i] * data.n)
    temp_1.append(data.condition_feat.repeat(data.n, 1))
    temp_2.append(data.condition_feat)

rxn_id = np.hstack(rxn_id)
temp_1 = torch.vstack(temp_1)
temp_2 = torch.vstack(temp_2)


In [7]:
temp_1.shape, temp_2.shape, rxn_id.shape

(torch.Size([141043, 1]), torch.Size([35729, 1]), (141043,))

In [18]:
GDSM[0].target_feat

AttributeError: 'GraphData' object has no attribute 'target_feat'